In [56]:
import numpy as np
import math
import csv
from proj1_helpers import *
from Hadrien import *
from ML_methods import *
from features_engineering import *
from correlation import *
import matplotlib.pyplot as plt


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[autoreload of ML_methods failed: Traceback (most recent call last):
  File "/Users/ana/anaconda3/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/Users/ana/anaconda3/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 368, in superreload
    module = reload(module)
  File "/Users/ana/anaconda3/lib/python3.6/imp.py", line 315, in reload
    return importlib.reload(module)
  File "/Users/ana/anaconda3/lib/python3.6/importlib/__init__.py", line 166, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 618, in _exec
  File "<frozen importlib._bootstrap_external>", line 674, in exec_module
  File "<frozen importlib._bootstrap_external>", line 781, in get_code
  File "<frozen importlib._bootstrap_external>", line 741, in source_to_code
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/Users/ana/Desktop/projet1/ML_hi

### load Data

In [4]:
y, tx, ids = load_csv_data("data_csv/train.csv", sub_sample=True)
print("y has shape {0}".format(y.shape))
print("X has shape {0}".format(tx.shape))

y has shape (5000,)
X has shape (5000, 30)


-> we can observe some (3 at all) different patterns of NAN values -> essayer de déterminer si ces values ont une corrélation avec le label:
- pattern 1: feature 0: 840 NAN -> 16.8% NAN
- pattern 2: features 23,24,25: 1985 NAN -> 39.7% NAN
- pattern 3: features 4,5,6,12,26,27,28: 3525 NAN -> 70.5 % NAN


Also :
- 562 samples de p1 font parti de p2.
- 734 samples de p1 font parti de p3.
- 0 samples de p2 font parti de p1.
- 1423 sample de p2 font parti de p3
- 0 samples de p3 font parti de p1.
- 0 samples de p3 font parti de p2.

Also:
- p2 inter p3 = 1985
- p1 union p3 union p2 = 3631

## Mets les outliers à NAN

In [5]:
tx_nan = transform_to_nan(tx, -900)

## Feature 22 engineering

In [23]:
txx = tx_nan.copy()

#ajoute des vecteurs booleens pour chaque valeur (0.0, 1.0, 2.0 ou 3.0) de la feature 22
tx0 = new_feature_PRI_jet_num(txx, 0.0)
tx1 = new_feature_PRI_jet_num(tx0, 1.0)
tx2 = new_feature_PRI_jet_num(tx1, 2.0)
tx3 = new_feature_PRI_jet_num(tx2, 3.0)

#enlever la colonne de la feature 22 with "delete(matrice, indice, colonne = 1)"
tx3_final = np.delete(tx3, 11, 1)

xx = tx3_final.copy()

## Divise DATAset depending on NAN patterns

In [24]:
from NAN_Pattern_Decomposition import decompose_dataset_according_to_NAN_pattern

p0, y0, p1, y1, p2, y2, p3, y3 = decompose_dataset_according_to_NAN_pattern(xx, y)

## Put NAN at mean

In [25]:
p0_nan_mean, mean =transform_to_mean(p0)
p1_nan_mean, mean =transform_to_mean(p1)
p2_nan_mean, mean =transform_to_mean(p2)
p3_nan_mean, mean =transform_to_mean(p3)
#warning parce qu'on a des colonnes remplies de NAN
print(p0_nan_mean.shape)

(2737, 33)


/Users/ana/Desktop/projet1/ML_higgs_project/ML_methods.py:99: RuntimeWarning: Mean of empty slice
  mean = np.nanmean(t, axis = 0)
/Users/ana/Desktop/projet1/ML_higgs_project/ML_methods.py:102: RuntimeWarning: Mean of empty slice
  column[np.isnan(column)] = np.nanmean(column)


# Feature Engineering

#### -> ajouter a la fin de la matrice de chaque groupe les differentes transformations

In [33]:
def add_feature_eng(dataset):
    
    tx_ = dataset.copy()
    features = list(range(30-1))   #-1 car on a enleve la colonne 22     

    # ajouter sin
    tx_sin = transform_feature_sin(tx_, features)
    print(np.shape(tx_sin))

    #ajouter cos
    tx_cos = transform_feature_cos(tx_sin, features)
    print(np.shape(tx_cos))

    #ajouter log
    tx_log = transform_feature_log(tx_cos, features)
    print(np.shape(tx_log))

    #ajouter x^2
    tx_pow2 = transform_feature_power(tx_log, features, 2)
    print(np.shape(tx_pow2))

    #ajouter x^3
    tx_pow3 = transform_feature_power(tx_pow2, features, 3)
    print(np.shape(tx_pow3))

    #ajouter sqrt
    tx_sqrt = transform_feature_sqrt(tx_pow3, features)
    print(np.shape(tx_sqrt))
    
    #standardize
    tx_stand, tx_mean, tx_srd = standardize_with_nan(tx_sqrt)
    
    dataset_eng = tx_stand.copy()
    
    return dataset_eng

# Ridge Regression

In [41]:
from train_model import *

### p0

In [55]:
p0_processed = add_feature_eng(p0_nan_mean)
#ridge_regression_exploration(y0, p0_processed, 0.8, np.logspace(-10, 10, 30))
#calcul de l'accuracy
ws = prediction_ridge_lambda_fixed(p0_processed, y0, 0.8, 1000, 0.00000001)
print(ws)

(2737, 62)
(2737, 91)
(2737, 120)
(2737, 149)
(2737, 178)
(2737, 207)


NameError: name 'prediction_ridge_lambda_fixed' is not defined